In [8]:

from google.colab import files

uploaded=files.upload()
for fn in uploaded.keys():
    print('user uploaded file "{name}" with length {length} bytes'.format(name=fn,length=len(uploaded[fn])))

Saving model (1).json to model (1).json
user uploaded file "model (1).json" with length 2586 bytes


In [2]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
text=str(open("song.txt",'rb').read())
text=text.lower()

In [0]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [9]:
print(len(characters))

40


In [0]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [0]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [0]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))

model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
model.fit(X_modified, Y_modified, epochs=100, batch_size=50)

model.save_weights('text_generator_400_0.2_400_0.2_400_0.2_100.h5')

Epoch 1/100
12629/12629 [==============================] - 145s 11ms/step - loss: 2.9449
Epoch 2/100
12629/12629 [==============================] - 142s 11ms/step - loss: 2.7724
Epoch 3/100
12629/12629 [==============================] - 142s 11ms/step - loss: 2.4437
Epoch 4/100
12629/12629 [==============================] - 141s 11ms/step - loss: 2.1877
Epoch 5/100
12629/12629 [==============================] - 142s 11ms/step - loss: 2.0423
Epoch 6/100
12629/12629 [==============================] - 143s 11ms/step - loss: 1.9146
Epoch 7/100
12629/12629 [==============================] - 144s 11ms/step - loss: 1.7608
Epoch 8/100
12629/12629 [==============================] - 141s 11ms/step - loss: 1.6068
Epoch 9/100
12629/12629 [==============================] - 141s 11ms/step - loss: 1.4571
Epoch 10/100
12629/12629 [==============================] - 143s 11ms/step - loss: 1.3024
Epoch 11/100
12629/12629 [==============================] - 143s 11ms/step - loss: 1.1448
Epoch 12/100
12629/

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [15]:
model.save_weights("model.h122")
print("Saved model to disk")

Saved model to disk


In [10]:
!ls

datalab  model (1).json  model.h122  sample_data  song.txt  ss.txt


In [0]:
from keras.models import model_from_json
json_file = open('model (1).json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [12]:
loaded_model.load_weights("model.h122")
print("Loaded model from disk")

Loaded model from disk


In [15]:
# Load LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import random
lines=open('song.txt','rb').readlines()
random.shuffle(lines)
open('ss.txt', 'wb').writelines(lines)
filename = "ss.txt"
raw_text = str(open(filename,'rb').read())
#print(raw_text)
#filename = "song.txt"
#raw_text = str(open(filename,'rb').read())
raw_text = raw_text.lower()


# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
model = Sequential()
model.add(LSTM(400, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))
#loaded_model.load_weights(filename)
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = loaded_model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  if (result==','):
    print('/n')
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print ("\nDone.")

Total Characters:  12729
Total Vocab:  40
Total Patterns:  12629
Seed:
" a paas mere palko pe rakh dun\r\ntera ishq mera fitoor hai\r\ntu mere dil ka karaar\r\naisa kahan mi "
lega tumhein\r\naachmi se mhri jaukee\r\nkyaar ho,/n
 tabaht ho jai\r\n\r\ntum mile kho gaya\r\nhai khud ka hi pata\r\nkya karun,/n
 kya nahi\r\nkuch bas mein na raha\r\nsamjhun kaise koi samjhaye\r\n\r\ndil kya kare jab kisi se\r\nkisi ko pyar ho jaaye\r\njaane kahan kab kisi ko\r\nkisi se pyaar ho jaaye\r\n\r\njaise pahan kati tuhhe\r\nde mere nazeeek tu\r\nye faas

KeyboardInterrupt: ignored